# Hands-on tutorial: Basic machine learning using with water

- **Notebook 1/3: Basic concepts and the three-point correlation function**

<hr>

- Material for the Erice [7th Workshop and School on Frontiers in Water Biophysics (FWB)](https://www.waterbiophysics.eu/Main/HomePage)
- EuMINe Training School on Machine Learning in Hydrated Biosystems.
- Sunday, July 6, 2025, Erice, Sicily, Italy.
  - Lecture: 8:45-9:55
  - Hands-on session: 14:45-15:45 (16:05)
- Material by [Mikko Karttunen](https://www.softsimu.net/mikko/) and [Matt Davies](https://www.researchgate.net/profile/Matthew-Davies-48).
- The full code and process will be available at GitHub later this summer (2025).
<hr>

## The approach

In this notenook, we will first review the background, and then go quickly through the computational proceess as described in the original references. We then return to the individual parts for more details in the other notebooks. The aim is to first obtain a feel of the outcomes and how data is represented with the hope that with that the individual parts become more transparent and applicable to other problems as well. In addition to water, in the following notebooks we will also use a simple lattice as an example since that allows the demonstration of some important properties before applying the methods to more complex systems.

## Learning outcomes
- Introduction to the $g_3$ correlation function
- Introduction to dimensional reduction
- Introduction to clustering

## The science behind this tutorial

The workflow in this hands-on tutorial is based on the paper,

 - Davies, M.; Reyes-Figueroa, A. D.; Gurtovenko, A. A.; Frankel, D.; Karttunen, M. Elucidating Lipid Conformations in the Ripple Phase: Machine Learning Reveals Four Lipid Populations. [Biophys. J. 2023, 122, 442–450](https://doi.org/10.1016/j.bpj.2022.11.024).

The above paper introduced a machine learning (ML) protocol for structure determination in hydrated lipid bilayers. The approach is completely general, and it is not limited to lipids. Here, we apply it to both a simple crystal structrure and water in the following notebooks. 

The $g_3$ three-point correlation function that is one of the key components for the analyses was originally introduced in the article, 

 - Sukhomlinov, S. V.; Müser, M. H. A Mixed Radial, Angular, Three-Body Distribution Function as a Tool for Local Structure Characterization: Application to Single-Component Structures. [J. Chem. Phys. 2020, 152, 194502](https://doi.org/10.1063/5.0007964).
  
The general protocol independent of the type of system is the following (we will go through the details in the other notebooks):

1. Read in trajectory data and topology
2. Choose the reference points for the $g_3$ correlation function and perform the $g_3$ analysis. The analysis produces countour plots in distance-angle (or, rather, distance-$\cos \theta$) coordinate system.
3. After that, the images are analyzed using the Structural Similarity Index Metric (SSIM) method, and dimensionality reduction using the t-SNE method is performed. As a result, a similarity matrix is produced.
4. Then, clustering using PCA/DBSCAN/HDBSCAN is performed.
5. Finally, one should analyze the clusters to verify that they properly represent well-defined structures.

## What is the $g_3$ three-point correlation function?

The three-body correlation function $g_3$  was introduced by Sukhomlinov and Müser in 2020 in

  - Sukhomlinov, S. V.; Müser, M. H. A Mixed Radial, Angular, Three-Body Distribution Function as a Tool for Local Structure Characterization: Application to Single-Component Structures. [J. Chem. Phys. 2020, 152, 194502](https://doi.org/10.1063/5.0007964).

It was originally applied to single component systems, but the original paper also demonstrates the method for NaCl. It can, however, be used for multicomponent systems including complex molecules such as polymers, lipid, and proteins. Typically, and virtually universally, for structure we measure the 2-point correlation function, or radial distribution function. For a single component system, it can be given as,

$$
\rho g^{(2)}(\vec{r}) = \frac{1}{N} \langle
\sum\limits_{i}^{N} \sum\limits_{j\ne i}^{N}
\delta(\vec{r}-\vec{r}_{ij})
\rangle
$$

For two different atom types $A$ and $B$ one can write,

$$
g_{AB}(r)=\frac{\langle \rho_B(r) \rangle}{\langle \rho_B \rangle_\mathrm{local}}=\frac{1}{\langle \rho_B \rangle_\mathrm{local}} \frac{1}{N_A} \sum_{i \in A}^{N_A} \sum_{j \in B}^{N_B} \frac{\delta(r_{ij}-r)}{4\pi r^2}.
$$

In the above, $\langle \rho _{B}(r)\rangle $ is the average density of type $B$ atoms at a distance $r$ from atoms $A$, the number of atoms of type is given by $N_A$, and the average density of atoms $B$ around atoms $A$ is given by $\langle \rho _B\rangle_\mathrm{local}$ is the atom density of type $B$ averaged around atoms $A$, with $r_{ij}$ being the distance between atoms $A$ and $B$, and \(\delta\) is the Dirac delta function.

The above 2-point correlation functions $g(r)$ depend only on the distance $r$. The 3-point correlation function adds a angle, thus increasing the dimensionality and resolution. It is also difficult to give it in a closed form.  In the figures below, the left one shows the idea of $g(r)$: only the distance is used. The right hand side demonstrates $g_3$ that use both a distance and an angle thus providing a higher resolution for measuring structure.

![alt text](./pics/atom_selection.png)
![alt text](./pics/g3_definition.svg)


The three-body distribution function  $g_3  \equiv g_3(r_\mathrm{BC}, \theta_\mathrm{ABC})$ is proportional to the probability of finding atom $\mathrm{C}$ at a distance $r_\mathrm{BC}$  from atom $\mathrm{B}$ atom given that atoms $\mathrm{A}$ and $\mathrm{B}$ are nearest neighbors  and $\theta_\mathrm{ABC} = \angle (\vec{r}_\mathrm{BC}, \vec{r}_\mathrm{BA})$. However, normalization is not straightforward in the same was as done with $g(r)$. The function $g_3$ contains information on the distribution of angles between a nearest-neighbor bond and a vector connecting a central atom with a more distant atom. Note that there are 2 *ad hoc* parameters: 

1. $d_\mathrm{AB}(r)$ which is characteristic bond length or most likely nearest-neighbor distance. 
2. cutoff $r_\mathrm{cut}$

While the traditional $g(r)$ captures the radial distribution of neighboring atoms around a central atom, the inclusion of the angle allows for determination of  *angular correlations*, and thus an accurate measurement the geometric arrangement of atomic triplets, or structural motifs. Such correlations are not included in $g(r)$. In addition to $g(r)$, it is common to use Voronoi tesellations for determining local structure. While Voronoi tesellation is a powerful and provides access distributions, $g_3$ allows arguably a more sensititve the detection of structural variations and changes between them. Arguably, from the physical perspective, $g_3$  provides a more complete description (than the typical methods) of the local structure by including angular correlations that are directly related to the directionality of interatomic forces. This is particularly important for systems where bond angles play a crucial role in determining stability and properties, such as covalently bonded materials, molecular crystals, and complex metallic phases.

### $g_3$ and Machine Learning

The $g_3$ method's ability to distinguish between *different local structural motifs* with *similar radial distributions* but *distinct angular arrangements* can be be used as a feature in ML as it can be applied as a sensitive measure for phase transitions, structural transformations, and the emergence of complex ordered phases. That was first done in 

   - Davies, M.; Reyes-Figueroa, A. D.; Gurtovenko, A. A.; Frankel, D.; Karttunen, M. Elucidating Lipid Conformations in the Ripple Phase: Machine Learning Reveals Four Lipid Populations. [Biophys. J. 2023, 122, 442–450](https://doi.org/10.1016/j.bpj.2022.11.024)

in the context of phase transitions in lipids bilayers. Other applications using the ML protocol introduced in the above paper include (at the time of writing) smoothing and restructuring of gold surfaces by N-heterocyclic carbenes, drug transport liposomes, and more complex lipid systems,

   - Goodwin, E.; Davies, M.; Bakiro, M.; Desroche, E.; Tumino, F.; Aloisio, M.; Crudden, C. M.; Ragogna, P. J.; Karttunen, M.; Barry, S. T. Atomic Layer Restructuring of Gold Surfaces by N-Heterocyclic Carbenes over Large Surface Areas. [ACS Nano 2025, 19, 15617–15626](https://doi.org/10.1021/acsnano.4c17517).
   -  Kehrein, J.; Gürsöz, E.; Davies, M.; Luxenhofer, R.; Bunker, A. Unravel the Tangle: Atomistic Insight into Ultrahigh Curcumin-Loaded Polymer Micelles. [Small 2023, e2303066](https://doi.org/10.1002/smll.202303066).
  - Heinonen, Suvi; Koivuniemi, Artturi; Davies, Matt; Karttunen, Mikko; Foged, Camilla; Bunker, Alex. Machine learning reveals structural characteristics of stereochemistry-specific interdigitation of synthetic monomycoloyl glycerol analogs. [JCIM, in press](https://doi.org/10.1021/acs.jcim.5c00615).
 

### Plotting $g_3$

The 3-point correlation function that is used as a metric here can be computed either intra- or inter-molecularly. For water, the obvious choice is inter-molecular. For larger and more flexible molecules such as lipids or polymers, it might be good to compute (also) intra-molecular $g_3$.



The figure below demonstrates the measurement of $g_3$:
- The first panel on the left identifies three atoms, $i$, $j$, and $k$ along the chain.
- The second figure from the left shows another possible configuration where the atom $k$ has moved to position $k^*$ that is rarely sampled.
- Then, a reference atom and its nearest neighbor are chosen,
- and finally the distance and angles with respect the reference atom and the  nearest neighbor are computed. The result is shown in the rightmost figure with the intensities corresponding to the probability of finding the atom at a given position and angle with respect to the reference. 

![alt text](./pics/g3_no_chain.svg)


Why $\cos \theta$ and not just the angle $\theta$? Because  the a priori probability  density  of  finding  another neighbor at a given value of $\
cos \theta$ is constant, while the one for finding another neighbor under an angle θ changes proportionally with $\sin \theta$. At the same time, we chose to let the $y$-axis go from negative (−1) to positive (+1) numbers as this makes the values of $\theta$ go from $0$ to $\pi$.

**Intra-molecular conformational clustering:** Let's assume that we have lipids as in the original paper. First, each lipid is isolated from the trajectory, and then $𝑔_3$ is calculated to quantify the intramolecular structure for each of them. After that, the SSIM of each lipid $𝑔_3$ distribution is compared with all other lipids. As the next step, embedding with t-SNE reduces the matrix of similarity values to a 2d form (from $𝑁_\mathrm{lipids}$ dimensions). This 2d data is then clustered (using, e.g., DBSCAN or HDBSCAN) in order to find if there are similar conformational groupings.

![alt text](./pics/g3_chain.svg)

### Why haven't I heard of these 3-point correlation functions?

Because 3-point correlation functions cannot be measured directly in diffraction experiments. 

### Computational efficiency

Since the calculation of $g_3$ involves atomic triplets (with a cutoff), there is a reasonanle computational cost. The computational complexity scales as $\mathcal{O}(N^2)$ for $N$ atoms within the cutoff ($r_\mathrm{cut}$).



### What is the Structural Similarity Index Metric (SSIM)
Defined as
$$
\mathrm{SSIM}(x,y) = \frac{\left( 2 \mu_x \mu_y + c_1 \right) \left(  2\sigma_{xy} + c_2\right)}
{\left( \mu_x^2 + \mu_y^2 + c_1 \right)  \left( \sigma_x^2 + \sigma_y^2 + c_2 \right)}, 
$$
SSIM determines the similarity of two images given by $x$ and $y$. The formula above combines 3 quantities to determine similarity, and $\mu_x$ is the window mean, $\sigma_x$ the variance of window $x$, and  $\sigma_{xy}$ the covariance of the two windows. The constant $c_i$ is a correction factor avoid division by zero. These quantities are evaluated *locally* as indicated by the formula above. Regarding its values, $\mathrm{SSIM} \in [0,1]$. The *global* SSIm value is given as
$$
\mathrm{SSIM} = \frac{1}{N}\sum \limits_{i=1}^{N}\mathrm{SSIM}(x_i,y_i).
$$
From imaging point of view, the SSIM is motivated by luminance, contrast and structure asthe carry information about the structure of the objects. Luminance masking is a phenomenon whereby image distortions (in this context) tend to be less visible in bright regions. Contrast masking is a phenomenon whereby distortions become less visible where there is significant activity or "texture" in the image. 


Note that one typical approach is to use the Mean Squared Error (MSE) to measure the similarity between images. The MSE, however, is a global measure and lacks the resolution of SSIM.

As a little fact of curiosity: The authors of the original SSIM paper were each accorded a Primetime Engineering Emmy Award in 2015 by the Television Academy since the SSIM subsequently has been adopted by the image processing community and in TV and social media industries.

### Dimensionality reduction: What is t-distributed Stochastic Neighbor Embedding (t-SNE)

The t-SNE method does dimensionality reduction, that is, it groups similar data and separates different ones. Other common methods include, for example, UMAP Principal Component Analysis (PCA). t-SNE was introduced by van der Maaten and Hinton in 2008,

   - Maaten, L.; Hinton, G. E. Visualizing Data Using T-SNE. Journal of Machine Learning Research 2008, 9, 2579–2605.

The t-SNE method is an unsupervised *non-linear* dimensionality reduction method. One of the main advantages is that t-SNE tries to preserve *local structure* whereas methods such as Principal Component Analysis (PCA) tries to find *global structure*. One should also keep in mind that with t-SNE, the distance between the clusers has no meaning, and the cluster size doesn't have a well-defined meaning.  

**Kullback–Leibler divergence:** a measure of how one probability distribution P is different from a second, reference probability distribution Q


#### Hyperparameters

- perplexity: measure for information that is defined as 2 to the power of the Shannon entropy. It controls the weight between the local and global.




  


### What is clustering?

#### Density-Based Spatial Clustering of Applications with Noise (DBSCAN)

DBSCAN is an *unsupervised* clustering method, that is, the number of clusters is not known beforehand. Importantly, it is *non-linear* meaning that it can find arbitrarily shaped clusters. DBSCAN has 2 hyperparameters:

1. $\varepsilon$: defines the neighborhood. A distance function is also needed.  The choice of distance function depends on the choice of $\varepsilon$, and has a major impact on the results. In general, it is necessary to first identify a reasonable measure of similarity for the data set, before the parameter ε can be chosen. There is no estimation for this parameter, but the distance functions needs to be chosen appropriately for the data set.

2. *minPTS*: defines the minimum number of points to form a *core*. This count includes also the point itself. A minimum value for *minPts* can be derived from the number of dimensions $D$ in the data set, as $\mathrm{minPts} \ge D + 1$. The value of $\mathrm{minPts} = 1$ does not make sense, as with this definition every point is a core point. With $\mathrm{minPts} \le 2$, the result will be the same as of hierarchical clustering with the single link metric, with the dendrogram cut at height $\varepsilon$. Therefore, $\mathrm{minPts} \ge 3$. Larger values are usually better for data sets with noise and will yield more significant clusters. As a rule of thumb, $\mathrm{minPts} = 2 \times D$ can be used.

The figure demonstrates how cluster points are classified. The figure on the right shows also two clusters that cannot be identified by linear methods. As discussed above, non-linear methods can can identify arbitrarily shaped clusters.

1. **core point:** a point is a core point if it has at least *minPTS* within *radius* $\varepsilon$. The point itself is counted in *minPTS*. In the figure, *minPTS = 3* and all red points have at least 3 points within a radius $\varepsilon$.
2. **border point:** a point is a bordr point if it has $2 < p < \mathrm{minPTS}$ points  within a radius $\varepsilon$.In the figure, that is the case for the orange points.
3. **noise point or outlier:** a point is an outlier if there are $p=1$ (the point itself) within a radius $\varepsilon$.

Furthermore:

- Each cluster must have at least one core point.
- a given point $q$ is called *directly reachable* from point $p$ if: 1) $p$ is a core point and the diastance from $q$ to $p$ <  $\varepsilon$.
- a given point $q$ is called *reachable*  from point $p$ if there is a connecting path between them. Note that this means that all the rest of the points with the possible exception of $q$ musct be core points.
- noise points are not reachable from any other point.

![alt text](./pics/dbscan_core_border_outlier.svg)
![alt text](./pics/dbscan_nonlinear.svg)

**How to determine the hyperparameters?**

**Computational cost**

The computational cost goes as $\mathcal{O}(N^2)$ or $\mathcal{O}(N \log N)$ depending on how indexing is done. The memory cost depends on if distance matrix is included: with distance matrix, the memory cost is $\mathcal{O}(N^2)$ and without, $\mathcal{O}(N)$.

